In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 텐서플로우의 로그 레벨을 설정하여 경고 메시지를 숨깁니다.

import tensorflow as tf  # 텐서플로우 라이브러리를 임포트합니다.
from tensorflow.keras import layers, Sequential  # 케라스에서 필요한 레이어와 Sequential 모델을 임포트합니다.
from tensorflow.keras.optimizers import Adam  # Adam 옵티마이저를 임포트합니다.
from tensorflow.keras.datasets import imdb  # IMDb 데이터셋을 임포트합니다.
from tensorflow.keras.preprocessing.sequence import pad_sequences  # 패딩 함수를 임포트합니다.

# 데이터셋을 로드하고 전처리하는 함수입니다.
def load_data(num_words, max_len):
    # imdb 데이터셋을 불러옵니다. 데이터셋에서 단어는 num_words 개를 가져옵니다.
    (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

    # 단어 개수가 다른 문장들을 Padding을 추가하여
    # 단어가 가장 많은 문장의 단어 개수로 통일합니다.
    X_train = pad_sequences(X_train, maxlen=max_len)  # X_train 데이터에 대해 패딩을 적용합니다.
    X_test = pad_sequences(X_test, maxlen=max_len)  # X_test 데이터에 대해 패딩을 적용합니다.
    
    return X_train, X_test, y_train, y_test  # 전처리된 데이터셋을 반환합니다.

# RNN 모델을 정의하는 함수입니다.
def build_rnn_model(num_words, embedding_len):
    model = Sequential()  # Sequential 모델을 초기화합니다.
    
    # 임베딩 레이어를 추가합니다.
    # input_dim: 단어 집합의 크기 (여기서는 num_words)
    # output_dim: 임베딩 벡터의 차원 (여기서는 embedding_len)
    model.add(layers.Embedding(input_dim=num_words, output_dim=embedding_len))
    
    # SimpleRNN 레이어를 추가합니다.
    # units: RNN의 유닛 수 (여기서는 16)
    model.add(layers.SimpleRNN(16))
    
    # Dense 레이어를 추가합니다.
    # units: 출력 유닛 수 (여기서는 1)
    # activation: 활성화 함수 (여기서는 'sigmoid' 사용)
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model  # 모델을 반환합니다.

# 메인 함수입니다.
def main(model=None, epochs=5):
    # IMDb 데이터셋에서 가져올 단어의 개수
    num_words = 6000
    
    # 각 문장이 가질 수 있는 최대 단어 개수
    max_len = 130
    
    # 임베딩 된 벡터의 길이
    embedding_len = 100
    
    # IMDb 데이터셋을 불러옵니다.
    X_train, X_test, y_train, y_test = load_data(num_words, max_len)
    
    if model is None:
        model = build_rnn_model(num_words, embedding_len)
    
    # 모델 학습을 위한 optimizer와 loss 함수를 설정합니다.
    # optimizer: Adam 옵티마이저를 사용하고, 학습률은 0.001로 설정합니다.
    optimizer = Adam(learning_rate=0.001)
    
    # 모델을 컴파일합니다.
    # loss: 이진 교차 엔트로피 손실 함수를 사용합니다.
    # metrics: 평가 지표로 정확도를 사용합니다.
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    # 모델을 학습합니다.
    # epochs: 전체 데이터셋을 몇 번 반복할지 설정합니다.
    # batch_size: 한 번에 몇 개의 샘플을 학습할지 설정합니다.
    # validation_split: 검증을 위해 학습 데이터의 몇 %를 사용할지 설정합니다.
    # shuffle: 데이터를 학습 전에 셔플할지 설정합니다.
    # verbose: 학습 중 출력할 로그의 레벨을 설정합니다.
    hist = model.fit(X_train, y_train, epochs=epochs, batch_size=100, validation_split=0.2, shuffle=True, verbose=2)
    
    # 모델을 테스트 데이터셋으로 테스트합니다.
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print()
    print("테스트 Loss: {:.5f}, 테스트 정확도: {:.3f}%".format(test_loss, test_acc * 100))
    
    return optimizer, hist  # 옵티마이저와 학습 기록을 반환합니다.

# 스크립트가 메인 프로그램으로 실행될 때 main() 함수를 호출합니다.
if __name__=="__main__":
    main()


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/5
200/200 - 4s - 18ms/step - accuracy: 0.6708 - loss: 0.5981 - val_accuracy: 0.7584 - val_loss: 0.5046
Epoch 2/5
200/200 - 3s - 15ms/step - accuracy: 0.8367 - loss: 0.3834 - val_accuracy: 0.8292 - val_loss: 0.4042
Epoch 3/5
200/200 - 3s - 15ms/step - accuracy: 0.9023 - loss: 0.2565 - val_accuracy: 0.8132 - val_loss: 0.4196
Epoch 4/5
200/200 - 3s - 15ms/step - accuracy: 0.9514 - loss: 0.1544 - val_accuracy: 0.8006 - val_loss: 0.4589
Epoch 5/5
200/200 - 3s - 16ms/step - accuracy: 0.9748 - loss: 0.0941 - val_accuracy: 0.8170 - val_loss: 0.4914

테스트 Loss: 0.50722, 테스트 정확도: 81.016%
